In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
# tf.enable_eager_execution()
import tensorflow_hub as hub
from tensorflow.keras import layers
import numpy as np
import PIL.Image as Image
import time

### 学習済みモデルの準備
https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2

In [ ]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

### 入力画像の準備
For this module, the size of the input image is fixed to height x width = 224 x 224 pixels. The input images are expected to have 3 RGB color values in the range [0,1], following the common image input conventions (analogously to TF 1.x).

In [ ]:
img_url = 'https://farm1.static.flickr.com/184/487161092_4c72f4c17a.jpg'
img_path = tf.keras.utils.get_file('cat.jpg', img_url, cache_dir='./')  # default ~/.keras
print(img_path)
img = Image.open(img_path)  # RGB
img

In [ ]:
IMAGE_SHAPE = (224, 224)
img = img.resize(IMAGE_SHAPE)
img = np.asarray(img) / 255.0
img = img[np.newaxis, ...]
img.shape

### 実行

In [ ]:
start = time.time()
result = classifier.predict(img)
elp = time.time() - start
print(elp)
(result.shape, result)

上位10件を表示

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt', cache_dir='./')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
def softmax(logits):
    exp = np.exp(logits)
    return exp / np.sum(exp)

In [ ]:
scores = softmax(result[0])
order = np.argsort(scores)[::-1]
for i in range(10):
    idx = order[i]
    print('%2d\t%s\t%.2f' % (i, imagenet_labels[idx], scores[idx]))

### 別のモデル
https://tfhub.dev/google/imagenet/resnet_v2_50/classification/3

In [ ]:
module = hub.Module('https://tfhub.dev/google/imagenet/resnet_v2_50/classification/3')
start = time.time()
logits = module(img)
elp = time.time() - start
print(elp)

In [ ]:
print(type(logits))
# tf.tensor to numpy
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    logits_arr = logits.eval()
scores = softmax(logits_arr[0])
order = np.argsort(scores)[::-1]
for i in range(10):
    idx = order[i]
    print('%2d\t%s\t%.2f' % (i, imagenet_labels[idx], scores[idx]))